In [27]:
%matplotlib ipympl
from QDP import qdp
import os.path
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt
from scipy import optimize
dp = '/home/ebert/jupyter-notebooks/FNode-data'

def expon(x, a, tau, b):
    return b + a*np.exp(-x/tau)

exclude = [2]

In [28]:
exp_name = ''
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
roi_map = ['cs', 'rb']
fmt_map = ['bo', 'rd']
cs_cuts = [[28],[3]]
rb_cuts = [[20],[3]]
marker_styles = [
    dict(color='cornflowerblue', marker='o', markeredgecolor='b', linestyle='None'),
    dict(color='indianred', marker='d', markeredgecolor='r', linestyle='None')
]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
q.generate_thresholds(loss=False)
print(q.get_thresholds())
retention = q.apply_thresholds(exclude_rois=exclude)
tbl_str = "ivar:\t{0}\tretention:{1} +- {2}"
entries = np.product(retention['retention'].shape[:-1])
for i in range(entries):
    print(tbl_str.format(
        retention['ivar'].reshape((entries,-1))[i][0],
        retention['retention'].reshape((entries,-1))[i],
        retention['error'].reshape((entries,-1))[i]
    ))
q.save_experiment_data()
q.experiments[0]['variable_list']
xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']

# plotting
fig, ax = plt.subplots(nrows=2, figsize=(5,8))
plt.subplots_adjust(wspace=0.3)
for i, yvals in enumerate(['retention', 'loading']):
    for e in range(len(retention['ivar'])):
        for r in range(2):            
            if yvals=='retention':
                try:
                    ax[i].errorbar(retention['ivar'][e, :, r], retention[yvals][e, :, r],
                        yerr=retention['error'][e,:,r],
                        **marker_styles[r]
                    )
                except Exception as ee:
                    print(ee)
                    ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], **marker_styles[r])
                # fitting
                xs = np.linspace(min(retention['ivar'][e, :, 0]), max(retention['ivar'][e, :, 0]),100)
                guess = [0.5, 10, 0.5]
                try:
                    popt, pcov = optimize.curve_fit(
                        expon,
                        retention['ivar'][e, :, 0],
                        retention[yvals][e, :, r],
                        p0=guess,
                        sigma=retention['error'][e, :, r]
                    )
                except Exception as ee:
                    print ee
                else:
                    ax[i].plot(xs, expon(xs, *popt), fmt_map[r][0]+'--', alpha=0.6)
                    print(popt)
                    fit_data = "{}\na: {:.3f}\ntau: {:.3f}\nb: {:.3f}"
                    ax[i].text(0.2+r*25, 0.1, fit_data.format(roi_map[r], *popt), fontsize=12)
            else:
                ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], **marker_styles[r])
                
        
    ax[i].set_ylim(0., 1.05*np.nanmax(retention[yvals][e,:,:2]))
    ax[i].set_xlabel(xlab)
    ax[i].set_ylabel(yvals)

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'exponential_retention_fit.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

data at: 2018_07_20/2018_07_20_10_37_28_csrb-hd-42-t1/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'gap_time']
--------------------
--------------------
--------------------
--------------------
On entry to DGESDD parameter number 5 had an illegal value
There may be some issue with your guess: `[  1.26882631e-17   0.00000000e+00   0.00000000e+00]`
--------------------
On entry to DGESDD parameter number 5 had an illegal value
There may be some issue with your guess: `[  1.26882631e-17   0.00000000e+00   0.00000000e+00]`
--------------------
{0: [[35], [2]], 1: [[31], [4]], 2: [[nan], [nan]]}
ivar:	8.5	retention:[ 0.95967742  0.89261745  1.        ] +- [ 0.02133057  0.02133057  0.02133057]
ivar:	246.8	retention:[ 0.7704918  0.86             inf] +- [ 0.02382668  0.02382668  0.02382668]
ivar:	485.1	retention:[ 0.78169014  0.79194631         inf] +- [ 0.02796652  0.02796652  0.02796652]
ivar:	723.4	retention:[ 0.71165644  0.82666667         nan]

FigureCanvasNbAgg()

[  0.50990395  10.80493506   0.72748986]
[  0.15953814  13.08261758   0.80930618]


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()


In [30]:
# plotting
fig, ax = plt.subplots(nrows=2, figsize=(5,8))
plt.subplots_adjust(wspace=0.3)
for i, yvals in enumerate(['retention', 'loading']):
    for e in range(len(retention['ivar'])):
        for r in range(2):            
            if yvals=='retention':
                try:
                    ax[i].errorbar(retention['ivar'][e, :, r], retention[yvals][e, :, r],
                        yerr=retention['error'][e,:,r],
                        fmt=fmt_map[r]
                    )
                except Exception as ee:
                    print(ee)
                    ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], fmt_map[r])
                # fitting
                xs = np.linspace(min(retention['ivar'][e, :, 0]), max(retention['ivar'][e, :, 0]),100)
                guess = [0.9, 1e4, 0]
                try:
                    popt, pcov = optimize.curve_fit(
                        lambda x, a, tau: expon(x, a, tau, 0.0),
                        retention['ivar'][e, :, 0],
                        retention[yvals][e, :, r],
                        p0=guess[:2],
                        sigma=retention['error'][e, :, r]
                    )
                except Exception as ee:
                    print ee
                else:
                    ax[i].plot(xs, expon(xs,popt[0],popt[1],0), fmt_map[r][0]+'--', alpha=0.6)
                    print(popt)
                    print(np.sqrt(np.diagonal(np.abs(pcov))))
                    fit_data = "{}\na: {:.3f}\ntau: {:.3f}\nb: {:.3f}"
                    ax[i].text(0.2+r*8e2, 0.1, fit_data.format(roi_map[r],popt[0],popt[1],0), fontsize=12)
            else:
                ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], fmt_map[r])
                
        
    ax[i].set_ylim(0.,1)
    ax[i].set_xlabel(xlab)
    ax[i].set_ylabel(yvals)

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'exponential_retention_fit.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

[  9.13393209e-01   3.21091310e+03]
[  3.69744369e-02   7.27996888e+02]
[  8.85771924e-01   7.73627645e+03]
[  1.48648694e-02   1.60369106e+03]


In [31]:
(1./3.21-1./9.7)**-1

4.797688751926041